In [33]:
!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib
from google.colab import auth
from googleapiclient.discovery import build
from google.auth import default

In [34]:
auth.authenticate_user()
credentials, _ = default()
service = build('docs', 'v1', credentials=credentials)
def create_document(title):
    document = service.documents().create(body={'title': title}).execute()
    print(f"Created document with ID: {document['documentId']}")
    return document['documentId']


In [35]:
def insert_text_and_style(service, document_id, text, styles, start_index=1):
    requests = []
    text += "\n"
    end_index = start_index + len(text)
    requests.append({
        'insertText': {
            'location': {'index': start_index},
            'text': text
        }
    })
    requests.append({
        'updateParagraphStyle': {
            'range': {'startIndex': start_index, 'endIndex': end_index},
            'paragraphStyle': {'namedStyleType': styles},
            'fields': 'namedStyleType'
        }
    })
    if "- [ ]" in text:
        requests.append({
            'createParagraphBullets': {
                'range': {'startIndex': start_index, 'endIndex': end_index},
                'bulletPreset': 'BULLET_CHECKBOX'
            }
        })

    if "@" in text:
        requests.append({
            'updateTextStyle': {
                'range': {'startIndex': start_index, 'endIndex': end_index},
                'textStyle': {'bold': True, 'foregroundColor': {
                    'color': {
                        'rgbColor': {'red': 1.0, 'green': 0.0, 'blue': 0.0}
                    }
                }},
                'fields': 'bold,foregroundColor'
            }
        })
    service.documents().batchUpdate(documentId=document_id, body={'requests': requests}).execute()
    return end_index

In [36]:
def format_document(document_id, markdown_text):
    lines = markdown_text.split('\n')
    index = 1
    for line in lines:
        if line.startswith('# '):  # Main title
            index = insert_text_and_style(service, document_id, line[2:], 'HEADING_1', index)
        elif line.startswith('## '):  # Section headers
            index = insert_text_and_style(service, document_id, line[3:], 'HEADING_2', index)
        elif line.startswith('### '):  # Sub-section headers
            index = insert_text_and_style(service, document_id, line[4:], 'HEADING_3', index)
        elif line.strip():  # Normal text, checks, and mentions
            index = insert_text_and_style(service, document_id, line, 'NORMAL_TEXT', index)

In [37]:
if __name__ == "__main__":
    doc_title = "Product Team Sync - May 15, 2023"
    doc_id = create_document(doc_title)
    markdown_text = """
# Product Team Sync - May 15, 2023

## Attendees
- Sarah Chen (Product Lead)
- Mike Johnson (Engineering)
- Anna Smith (Design)
- David Park (QA)

## Agenda

### 1. Sprint Review
* Completed Features
  * User authentication flow
  * Dashboard redesign
  * Performance optimization
    * Reduced load time by 40%
    * Implemented caching solution
* Pending Items
  * Mobile responsive fixes
  * Beta testing feedback integration

### 2. Current Challenges
* Resource constraints in QA team
* Third-party API integration delays
* User feedback on new UI
  * Navigation confusion
  * Color contrast issues

### 3. Next Sprint Planning
* Priority Features
  * Payment gateway integration
  * User profile enhancement
  * Analytics dashboard
* Technical Debt
  * Code refactoring
  * Documentation updates

## Action Items
- [ ] @sarah: Finalize Q3 roadmap by Friday
- [ ] @mike: Schedule technical review for payment integration
- [ ] @anna: Share updated design system documentation
- [ ] @david: Prepare QA resource allocation proposal

## Next Steps
* Schedule individual team reviews
* Update sprint board
* Share meeting summary with stakeholders

## Notes
* Next sync scheduled for May 22, 2023
* Platform demo for stakeholders on May 25
* Remember to update JIRA tickets

---
Meeting recorded by: Sarah Chen
Duration: 45 minutes
"""
format_document(doc_id, markdown_text)

Created document with ID: 1Uenool2uo1b0Z1OROCvqxDRE3qfhVyZwez1DjwYyJSk
